# Impacto Humano y Daño de Terremotos (Earthquake)

## Importar librerías

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import re
from geopy.geocoders import Nominatim
import folium
from time import sleep

In [3]:
geolocator = Nominatim(user_agent="MozillaFirefox", timeout=10)

## Cargar base de datos

03Limpio/03desastreslimpio.csv

In [4]:
df = pd.read_csv('../../Data/03Limpio/03desastreslimpio.csv')
df.head(10)

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
0,1900-9002-CPV,1900,9002,Climatological,Drought,Drought,Cabo Verde,CPV,Western Africa,Africa,...,NaN,1900,0,0,1900,0,0,11000,0,0
1,1900-9001-IND,1900,9001,Climatological,Drought,Drought,India,IND,Southern Asia,Asia,...,NaN,1900,0,0,1900,0,0,1250000,0,0
2,1902-0012-GTM,1902,12,Geophysical,Earthquake,Ground movement,Guatemala,GTM,Central America,Americas,...,-91,1902,4,18,1902,4,18,2000,0,843726
3,1902-0003-GTM,1902,3,Geophysical,Volcanic activity,Ash fall,Guatemala,GTM,Central America,Americas,...,NaN,1902,4,8,1902,4,8,1000,0,0
4,1902-0010-GTM,1902,10,Geophysical,Volcanic activity,Ash fall,Guatemala,GTM,Central America,Americas,...,NaN,1902,10,24,1902,10,24,6000,0,0
5,1903-0006-CAN,1903,6,Geophysical,Mass movement (dry),Rockfall,Canada,CAN,Northern America,Americas,...,NaN,1903,4,29,1903,4,29,76,23,0
6,1903-0012-COM,1903,12,Geophysical,Volcanic activity,Ash fall,Comoros (the),COM,Eastern Africa,Africa,...,NaN,1903,0,0,1903,0,0,17,0,0
7,1904-0003-BGD,1904,3,Meteorological,Storm,Tropical cyclone,Bangladesh,BGD,Southern Asia,Asia,...,NaN,1904,11,0,1904,11,0,0,0,0
8,1905-0005-CAN,1905,5,Geophysical,Mass movement (dry),Rockfall,Canada,CAN,Northern America,Americas,...,NaN,1905,8,13,1905,8,13,18,18,0
9,1905-0003-IND,1905,3,Geophysical,Earthquake,Ground movement,India,IND,Southern Asia,Asia,...,76.16,1905,4,4,1905,4,4,20000,0,812477


## EDA Inicial Básico

In [5]:
df.shape

(16636, 26)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16636 entries, 0 to 16635
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Dis No             16636 non-null  object
 1   Year               16636 non-null  int64 
 2   Seq                16636 non-null  int64 
 3   Disaster Subgroup  16636 non-null  object
 4   Disaster Type      16636 non-null  object
 5   Disaster Subtype   13313 non-null  object
 6   Country            16636 non-null  object
 7   ISO                16636 non-null  object
 8   Region             16636 non-null  object
 9   Continent          16636 non-null  object
 10  Location           14825 non-null  object
 11  Origin             4085 non-null   object
 12  Associated Dis     3593 non-null   object
 13  Dis Mag Value      16636 non-null  int64 
 14  Dis Mag Scale      15416 non-null  object
 15  Latitude           2775 non-null   object
 16  Longitude          2775 non-null   objec

## Filtro Disaster Type == Earthquake

In [7]:
filter = df['Disaster Type'] == 'earthquake'
df_earthquake = df[filter]

In [8]:
df_earthquake.head(2)

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj


In [9]:
df_earthquake.shape

(0, 26)

In [10]:
df_earthquake['Disaster Type'].dtype

dtype('O')

# Limpieza de coordenadas

## Función para limpiar coordenadas filtrado para df_earthquake

In [11]:
# Convertir 'Latitude' y 'Longitude' a string
df_earthquake['Latitude'] = df_earthquake['Latitude'].astype(str)
df_earthquake['Longitude'] = df_earthquake['Longitude'].astype(str)

# Limpiar las coordenadas de latitud y longitud
df_earthquake['Latitude'] = df_earthquake['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x))
df_earthquake['Longitude'] = df_earthquake['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x))

# Quitar puntos finales
df_earthquake['Latitude'] = df_earthquake['Latitude'].str.rstrip('.')
df_earthquake['Longitude'] = df_earthquake['Longitude'].str.rstrip('.')

# Añadir valores anómalos de latitud y longitud a listas
anomalous_lat = []
anomalous_lon = []

def convert_coordinates(x, convert_nan=True):
    if not x:
        return np.nan

    # Remover caracteres 'N' y 'E'
    x = x.replace(' N', '').replace(' E', '')
    
    # Remover puntos extras en decimales
    x = re.sub('^(\d+\.\d{2})\..*', r'\1', x)

    # Conversión a negativo para S y W
    try:
        value = float(x)
        if x[-1] == 'S' or x[-1] == 'W':
            return -value
        else:
            return value
    except ValueError:
        return np.nan

# Convertir a float
df_earthquake['Latitude'] = df_earthquake['Latitude'].apply(convert_coordinates)
df_earthquake['Longitude'] = df_earthquake['Longitude'].apply(convert_coordinates)

# Redondear decimales
df_earthquake['Latitude'] = df_earthquake['Latitude'].round(2)
df_earthquake['Longitude'] = df_earthquake['Longitude'].round(2)

# Identificación de valores anómalos fuera de rango de grados de sistema de coordenadas
for index, row in df_earthquake.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']

    if latitude < -90 or latitude > 90:
        anomalous_lat.append(latitude)

    if longitude < -180 or longitude > 180:
        anomalous_lon.append(longitude)

print("Valores anómalos de Latitude:", anomalous_lat)
print("Valores anómalos de Longitude:", anomalous_lon)


Valores anómalos de Latitude: []
Valores anómalos de Longitude: []


In [12]:
# Función para rellenar los valores en anomalous_lat y anomalous_lon
def fill_anomalous_latlon(row):
    if row['Latitude'] in anomalous_lat and row['Longitude'] in anomalous_lon:
        if pd.notnull(row['Location']):
            location = geolocator.geocode(row['Location'])
        else:
            location = geolocator.geocode(row['Country'])

        if location:
            row['Latitude'] = location.latitude
            row['Longitude'] = location.longitude
            sleep(1)
    return row

# Crear un objeto geolocator
geolocator = Nominatim(user_agent='limpiar_anomalos')

# Aplicar la función fill_anomalous_latlon al DataFrame
df_earthquake_filled_latlon = df_earthquake.apply(fill_anomalous_latlon, axis=1)
df_earthquake_filled_latlon.head()

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj


1. Definimos la función fill_anomalous_latlon que toma una fila como entrada. Si los valores de latitud y longitud de la fila se encuentran en las listas anomalous_lat y anomalous_lon, respectivamente, la función intenta geocodificar la ubicación mediante la columna Location o la columna Country. Si se encuentra una ubicación válida, se actualizan los valores de latitud y longitud de la fila.

2. La llamada a la función sleep(1) agrega un retraso de 1 segundo entre las solicitudes de geolocalización para cumplir con la política de uso del servicio de geocodificación.

3. Luego creamos un objeto geolocalizador utilizando el geocodificador Nominatim de la biblioteca geopy.

4. Finalmente, aplicamos la función fill_anomalous_latlon a cada fila del DataFrame usando el método apply con axis=1, lo que indica que la función debe aplicarse por filas.

In [13]:
# def fill_anomalos_lat_lon(row):
#     if abs(row['Latitude'])>90 and abs(row['Longitude'])>180:
    
#         location = geolocator.geocode(row['Country'])
#         if location:
#             row['Latitude'] = location.latitude
#             row['Longitude'] = location.longitude
#             sleep(1)
#     return row

# df_earthquake_fillanomalos = df_earthquake(fill_anomalos_lat_lon, axis=1)
# df_earthquake_fillanomalos.head()

## Tests de verificación de limpieza de coordenadas

In [14]:
df_earthquake[['Latitude', 'Longitude', 'Country']][700:800:]

,Latitude,Longitude,Country


In [15]:
print(df_earthquake['Latitude'])
print(df_earthquake['Longitude'])

Series([], Name: Latitude, dtype: object)
Series([], Name: Longitude, dtype: object)


In [16]:
print(df_earthquake['Latitude'].dtype)
print(df_earthquake['Longitude'].dtype)

object
object


In [17]:
# # Examinar filas específicas para verificar si el proceso de limpieza ha manejado los valores anómalos correctamente.
# print(df_earthquake.loc[12000, 'Latitude'])
# print(df_earthquake.loc[12000, 'Longitude'])

In [18]:
# Valores nulos de Longitude y Latitude
df_earthquake[['Longitude', 'Latitude']].isnull().sum()

Longitude    0.0
Latitude     0.0
dtype: float64

## Valores Anómalos de Latitud y Longitud

In [19]:
len(anomalous_lat)

0

In [20]:
len(anomalous_lon)

0

#### Índices Anómalos de Latitud y longitud

In [21]:
anomalous_lat_idx = []
for index, lat in enumerate(anomalous_lat):
    if lat < -90 or lat > 90:
        anomalous_lat_idx.append(index)

print("Índices Anómalos de Latitud:", anomalous_lat_idx)

Índices Anómalos de Latitud: []


In [22]:
anomalous_lon_idx = []
for index, lon in enumerate(anomalous_lon):
    if lat < -90 or lat > 90:
        anomalous_lon_idx.append(index)

print("Índices Anómalos de Longitud:", anomalous_lon_idx)

Índices Anómalos de Longitud: []


## Outliers de Latitude y Longitude

In [23]:
# sns.boxplot(df['Longitude'])

In [24]:
# sns.boxplot(df['Latitude'])

## Valores faltantes

In [25]:
df_earthquake.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Dis No             0 non-null      object
 1   Year               0 non-null      int64 
 2   Seq                0 non-null      int64 
 3   Disaster Subgroup  0 non-null      object
 4   Disaster Type      0 non-null      object
 5   Disaster Subtype   0 non-null      object
 6   Country            0 non-null      object
 7   ISO                0 non-null      object
 8   Region             0 non-null      object
 9   Continent          0 non-null      object
 10  Location           0 non-null      object
 11  Origin             0 non-null      object
 12  Associated Dis     0 non-null      object
 13  Dis Mag Value      0 non-null      int64 
 14  Dis Mag Scale      0 non-null      object
 15  Latitude           0 non-null      object
 16  Longitude          0 non-null      object
 17  Start Yea

In [26]:
df_earthquake.isnull().sum()

Dis No               0.0
Year                 0.0
Seq                  0.0
Disaster Subgroup    0.0
Disaster Type        0.0
Disaster Subtype     0.0
Country              0.0
ISO                  0.0
Region               0.0
Continent            0.0
Location             0.0
Origin               0.0
Associated Dis       0.0
Dis Mag Value        0.0
Dis Mag Scale        0.0
Latitude             0.0
Longitude            0.0
Start Year           0.0
Start Month          0.0
Start Day            0.0
End Year             0.0
End Month            0.0
End Day              0.0
Total Deaths         0.0
Total Affected       0.0
Total Damages Adj    0.0
dtype: float64

# Afectados por terremotos

### Muertos por Terremotos:

7.004.082

Siete millones, cuatro mil, ochenta y dos.

In [27]:
df_earthquake_deaths = df_earthquake['Total Deaths'].sum()

### Afectados por terremotos:
3.968.190.458

Tres billones, novescientos sesenta y ocho millones, ciento noventa mil cuatrocientos cincuenta y ocho.

o

Tres mil novecientos sesenta y ocho millones, ciento noventa mil, cuatrocientos cincuenta y ocho.

In [28]:
df_earthquake_affected = df_earthquake['Total Affected'].sum()

## Daños por terremotos

In [29]:
df_earthquake_damages = df_earthquake['Total Damages Adj'].sum()

## Análisis geográfico de muertos y afectados

### Muertos y afectados por continente

In [30]:
df_earthquake_continent_counts = df_earthquake['Continent'].value_counts()
df_earthquake_continent_counts

Series([], Name: Continent, dtype: int64)

In [31]:
# Check for missing values in 'Total Deaths'
missing_values = df_earthquake['Total Deaths'].isnull().sum()
print("Number of missing values in 'Total Deaths':", missing_values)

# Drop rows with missing values in 'Total Deaths'
df_earthquake_cleaned = df_earthquake.dropna(subset=['Total Deaths'])



Number of missing values in 'Total Deaths': 0


In [32]:
# Check unique values in 'Total Deaths' column
unique_values = df_earthquake['Total Deaths'].unique()
print("Unique values in 'Total Deaths' column:", unique_values)


Unique values in 'Total Deaths' column: []


In [33]:
# Plot the bar chart
plt.figure(figsize=(10, 6))
sns.barplot(data=df_earthquake, x='Continent', y='Total Deaths')
plt.title('Comparativa de Muertes Totales en Terremotos por Continente')
plt.xlabel('Terremotos por Continente')
plt.ylabel('Muertes totales')
plt.show()


ValueError: min() arg is an empty sequence

<Figure size 1000x600 with 0 Axes>

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df_earthquake, x='Continent', y='Total Affected')
plt.title('Comparativa de Total de Afectados en Terremotos por Continente')
plt.xlabel('Terremotos por Continente')
plt.ylabel('Muertes totales')
plt.show()

Asia es el continente con más muertos y afectados respecto a otros.

### Muertos y afectados por región

In [ ]:
df_earthquake_region_counts = df_earthquake['Region'].value_counts()
df_earthquake_region_counts

Series([], Name: Region, dtype: int64)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df_earthquake, x='Region', y='Total Deaths')
plt.title('Comparativa de Muertes Totales en Terremotos por Región')
plt.xlabel('Terremotos por Continente')
plt.ylabel('Muertes totales')
plt.xticks(rotation=75)
plt.show()

ValueError: min() arg is an empty sequence

<Figure size 1000x600 with 0 Axes>

El Este Asiático supera por creces el número de muertes por terremotos.

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df_earthquake, x='Region', y='Total Affected')
plt.title('Comparativa de Total de Afectados en Terremotos por Región')
plt.xlabel('Terremotos por Región')
plt.ylabel('Afectados totales')
plt.xticks(rotation=75)
plt.show()

El Este y Sur de Asia tienen altos niveles de personas afectadas por terremotos, seguidos de lejos por el Sudeste Asiático.

### Muertos y afectados por país

In [ ]:
df_earthquake_country_counts = df_earthquake['Country'].value_counts()
df_earthquake_country_counts

#### Total de Muertes en Países Agrupados por Regiones

In [ ]:
# Agrupar por región y calcular la suma total de muertes por cada país
region_country_deaths = df_earthquake.groupby(['Region', 'Country'])['Total Deaths'].sum().reset_index()

# Ordenar por Region y Total Deaths
region_country_deaths = region_country_deaths.sort_values(['Region', 'Total Deaths'], ascending=[True, False])

# Unique de regiones
regions = region_country_deaths['Region'].unique()

# Crear barplot separado por cada región
for region in regions:
    plt.figure(figsize=(10, 6))
    region_data = region_country_deaths[region_country_deaths['Region'] == region]
    sns.barplot(data=region_data, x='Country', y='Total Deaths')
    plt.title(f'Muertes Totales en Terremotos - {region}')
    plt.xlabel('País')
    plt.ylabel('Muertes Totales')
    plt.xticks(rotation=75)
    plt.tight_layout()
    plt.show()

#### Total de Afectados en Países Agrupados por Regiones

In [ ]:
# Agrupar por región y calcular la suma total de afectados por cada país
region_country_affected = df_earthquake.groupby(['Region', 'Country'])['Total Affected'].sum().reset_index()

# Ordenar por Region y Total Deaths
region_country_affected = region_country_affected.sort_values(['Region', 'Total Affected'], ascending=[True, False])

# Unique de regiones
regions = region_country_affected['Region'].unique()

# Crear barplot separado por cada región
for region in regions:
    plt.figure(figsize=(10, 6))
    region_data = region_country_affected[region_country_affected['Region'] == region]
    sns.barplot(data=region_data, x='Country', y='Total Affected')
    plt.title(f'Afectados Totales en Terremotos - {region}')
    plt.xlabel('País')
    plt.ylabel('Afectados Totales')
    plt.xticks(rotation=75)
    plt.tight_layout()
    plt.show()

#### Total de Afectados en Países Agrupados por Regiones

In [ ]:
# Agrupar por región y calcular la suma total de daños por cada país
region_country_damages = df_earthquake.groupby(['Region', 'Country'])['Total Damages Adj'].sum().reset_index()

# Ordenar por Region y Total Damages Adj
region_country_damages = region_country_damages.sort_values(['Region', 'Total Damages Adj'], ascending=[True, False])

# Unique de regiones
regions = region_country_damages['Region'].unique()

# Crear barplot separado por cada región
for region in regions:
    plt.figure(figsize=(10, 6))
    region_data = region_country_damages[region_country_damages['Region'] == region]
    sns.barplot(data=region_data, x='Country', y='Total Damages Adj')
    plt.title(f'Daños Totales en Terremotos - {region}')
    plt.xlabel('País')
    plt.ylabel('Daños Totales')
    plt.xticks(rotation=75)
    plt.tight_layout()
    plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df_earthquake, x='Country', y='Total Affected')
plt.title('Comparativa de Total de Afectados en Terremotos por País')
plt.xlabel('Terremotos por País')
plt.ylabel('Afectados totales')
plt.xticks(rotation=75)
plt.show()

## Análisis de región y origen de desastre

### Total de muertes por origen en las distintas regiones

In [ ]:
# Agrupar por región y calcular la suma total de afectados por cada origen
region_origin_deaths = df_earthquake.groupby(['Region', 'Origin'])['Total Affected'].sum().reset_index()

# Ordenar por Región, Origen y Total Deaths
region_origin_deaths = region_origin_deaths.sort_values(['Region', 'Total Affected'], ascending=[True, False])

# Unique de regiones
regions = region_origin_deaths['Region'].unique()

# Crear barplot separado por cada región
for region in regions:
    plt.figure(figsize=(10, 6))
    region_data = region_origin_deaths[region_origin_affected['Region'] == region]
    sns.barplot(data=region_data, x='Origin', y='Total Affected')
    plt.title(f'Muertes en terremotos según origen- {region}')
    plt.xlabel('Origen')
    plt.ylabel('Muertes Totales')
    plt.xticks(rotation=75)
    plt.show()

### Total de afectados por origen en las distintas regiones

In [ ]:
# Agrupar por región y calcular la suma total de afectados por cada origen
region_origin_affected = df_earthquake.groupby(['Region', 'Origin'])['Total Affected'].sum().reset_index()

# Ordenar por Región, Origen y Total Deaths
region_origin_affected = region_origin_affected.sort_values(['Region', 'Total Affected'], ascending=[True, False])

# Unique de regiones
regions = region_origin_affected['Region'].unique()

# Crear barplot separado por cada región
for region in regions:
    plt.figure(figsize=(10, 6))
    region_data = region_origin_affected[region_origin_affected['Region'] == region]
    sns.barplot(data=region_data, x='Origin', y='Total Affected')
    plt.title(f'Afectados Totales en Terremotos - {region}')
    plt.xlabel('Origen')
    plt.ylabel('Afectados Totales')
    plt.xticks(rotation=75)
    plt.show()

### Total de daños por origen en las distintas regiones

In [ ]:
# Agrupar por región y calcular la suma total de daños por cada origen
region_origin_damages = df_earthquake.groupby(['Region', 'Origin'])['Total Affected'].sum().reset_index()

# Ordenar por Región, Origen y Total Damages
region_origin_damages = region_origin_damages.sort_values(['Region', 'Total Affected'], ascending=[True, False])

# Unique de regiones
regions = region_origin_damages['Region'].unique()

# Crear barplot separado por cada región
for region in regions:
    plt.figure(figsize=(10, 6))
    region_data = region_origin_damages[region_origin_damages['Region'] == region]
    sns.barplot(data=region_data, x='Origin', y='Total Affected')
    plt.title(f'Daños por Origen en Terremotos - {region}')
    plt.xlabel('Origen')
    plt.ylabel('Daños Totales')
    plt.xticks(rotation=75)
    plt.show()

## Análisis de región y desastre asociado

In [ ]:
# Agrupar por región y calcular la suma total de muertes por cada desastre asociado
region_assodis_deaths = df_earthquake.groupby(['Region', 'Associated Dis'])['Total Deaths'].sum().reset_index()

# Ordenar por Región, Associated Disaster y Total Deaths
region_assodis_deaths = region_assodis_deaths.sort_values(['Region', 'Total Affected'], ascending=[True, False])

# Unique de regiones
regions = region_assodis_deaths['Region'].unique()

# Crear barplot separado por cada región
for region in regions:
    plt.figure(figsize=(10, 6))
    region_data = region_assodis_deaths[region_assodis_deaths['Region'] == region]
    sns.barplot(data=region_data, x='Associated Dis', y='Total Deaths')
    plt.title(f'Muertes y Desastre Asociado en Terremotos - {region}')
    plt.xlabel('Desastre Asociado')
    plt.ylabel('Muertes Totales')
    plt.xticks(rotation=75)
    plt.show()

In [ ]:
# Agrupar por región y calcular la suma total de afectados por cada desastre asociado
region_assodis_affected = df_earthquake.groupby(['Region', 'Associated Dis'])['Total Affected'].sum().reset_index()

# Ordenar por Región, Associated Disaster y Total Affected
region_assodis_affected = region_assodis_affected.sort_values(['Region', 'Total Affected'], ascending=[True, False])

# Unique de regiones
regions = region_assodis_affected['Region'].unique()

# Crear barplot separado por cada región
for region in regions:
    plt.figure(figsize=(10, 6))
    region_data = region_assodis_affected[region_assodis_affected['Region'] == region]
    sns.barplot(data=region_data, x='Associated Dis', y='Total Affected')
    plt.title(f'Afectados y Desastre Asociado en Terremotos - {region}')
    plt.xlabel('Desastre Asociado')
    plt.ylabel('Afectados Totales')
    plt.xticks(rotation=75)
    plt.show()

In [ ]:
# Agrupar por región y calcular la suma total de daños por cada desastre asociado
region_assodis_damages = df_earthquake.groupby(['Region', 'Associated Dis'])['Total Damages Adj'].sum().reset_index()

# Ordenar por Región, Associated Disaster y Total Damages
region_assodis_damages = region_assodis_damages.sort_values(['Region', 'Total Damages Adj'], ascending=[True, False])

# Unique de regiones
regions = region_assodis_damages['Region'].unique()

# Crear barplot separado por cada región
for region in regions:
    plt.figure(figsize=(10, 6))
    region_data = region_assodis_damages[region_assodis_damages['Region'] == region]
    sns.barplot(data=region_data, x='Associated Dis', y='Total Damages Adj')
    plt.title(f'Daños y Desastre Asociado en Terremotos - {region}')
    plt.xlabel('Desastre Asociado')
    plt.ylabel('Daños Totales')
    plt.xticks(rotation=75)
    plt.show()

### DataFrames de Muertes, Afectados y Daños por Continente, Región y Países

In [ ]:
deaths_by_continent = df_earthquake.groupby('Continent')[['Total Deaths', 'Total Affected', 'Total Damages Adj']].sum().reset_index()
# deaths_by_continent['Total Affected'] = deaths_by_continent['Total Affected'].astype(int)  # Convertir a int
deaths_by_continent

In [ ]:
deaths_by_region = df_earthquake.groupby('Region')['Total Deaths', 'Total Affected', 'Total Damages Adj'].sum().reset_index()
deaths_by_region

In [ ]:
total_deaths_by_country = df_earthquake.groupby('Country')['Total Deaths', 'Total Affected', 'Total Damages Adj'].sum().reset_index()
total_deaths_by_country

In [ ]:
deaths_by_country = df_earthquake.groupby('Country')['Total Deaths'].value_counts()
deaths_by_country

In [ ]:
affected_by_country = df_earthquake.groupby('Country')['Total Affected'].value_counts()
affected_by_country

# Visualizaciones

In [ ]:
df_earthquake.columns

## Heatmaps

### Segmentación de correlaciones: por tipo, geográfica y temporal

In [ ]:
df_earthquake_corr_type = df_earthquake[['Disaster Subtype', 'Associated Dis']]
df_earthquake_corr_geo = df_earthquake[['Continent', 'Region', 'Country', 'Location', 'Origin']]
df_earthquake_corr_temp = df_earthquake[['Start Day', 'End Day', 'Start Month', 'End Month', 'Start Year', 'End Year']]
df_earthquake_corr_magn = df_earthquake[['Dis Mag Value', 'Dis Mag Scale']]
df_earthquake_corr_geoloc = df_earthquake[['Latitude', 'Longitude']]

In [ ]:
# Plot de heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df_earthquake.corr(), annot=True, cmap='Blues', cbar=False)
plt.title('Heatmap de correlaciones en df_earthquake')
plt.show()

### Correlaciones por tipo de desastre

In [ ]:
for column in df_earthquake_corr_type:
    crosstab = pd.crosstab(df_earthquake_corr_type[column], columns='count')
    plt.figure(figsize=(8, 6))
    sns.heatmap(crosstab, annot=True, cmap='Blues', cbar=False)
    plt.title(f"Heatmap de {column}")
    plt.xlabel('')
    plt.ylabel('')
    plt.show()

### Correlaciones geográficas

In [ ]:
for column in df_earthquake_corr_geo:
    crosstab = pd.crosstab(df_earthquake_corr_geo[column], columns='count')
    plt.figure(figsize=(8, 6))
    sns.heatmap(crosstab, annot=True, cmap='Blues', cbar=False)
    plt.title(f"Heatmap de {column}")
    plt.xlabel('')
    plt.ylabel('')
    plt.show()

### Correlaciones temporales

In [ ]:
for column in df_earthquake_corr_temp:
    crosstab = pd.crosstab(df_earthquake_corr_temp[column], columns='count')
    plt.figure(figsize=(8, 6))
    sns.heatmap(crosstab, annot=True, cmap='Blues', cbar=False)
    plt.title(f"Heatmap de {column}")
    plt.xlabel('')
    plt.ylabel('')
    plt.show()

In [ ]:
origin_stats = df_earthquake.groupby('Origin').agg({
    'Disaster Subtype': 'count',  # Total de Terremotos
    'Total Deaths': 'sum',
    'Total Affected': 'sum'
}).reset_index()

# Ordenar por Total Deaths
origin_stats = origin_stats.sort_values('Total Deaths', ascending=False, inplace=True)
origin_stats

# Pivot Tables

## Profundización de Análisis de Impacto Geográfico con Pivot Tables

In [ ]:
impact_order = ['Total Deaths', 'Total Affected', 'Total Damages Adj']

### Impacto por subtipo de desastre

#### Por país

In [ ]:
df_earthquake_impact_country = df_earthquake.dropna(subset=['Country', 'Total Deaths', 'Total Affected', 'Total Damages Adj'])
df_earthquake_impact_country_pivot = df_earthquake_impact_country.pivot_table(
    index='Country',
    columns='Disaster Subtype',
    values=['Total Deaths', 'Total Affected', 'Total Damages Adj'],
    aggfunc='sum',
    fill_value=0
)
df_earthquake_impact_country_pivot

In [ ]:
df_earthquake_impact_country_pivot.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('Comparativa de Impacto por País')
plt.xlabel('País')
plt.ylabel('Conteo Total')
plt.legend(title='Subtipo de Desastre')
plt.xticks(rotation=75)
plt.show()


#### Por Location

In [ ]:
df_earthquake_impact_location = df_earthquake.dropna(subset=['Location', 'Total Deaths', 'Total Affected', 'Total Damages Adj'])
df_earthquake_impact_location_pivot = df_earthquake_impact_location.pivot_table(
    index='Location',
    columns='Disaster Subtype',
    values=['Total Deaths', 'Total Affected', 'Total Damages Adj'],
    aggfunc='sum',
    fill_value=0
)
df_earthquake_impact_location_pivot

#### Por Origin

In [ ]:
df_earthquake_impact_origin = df_earthquake.dropna(subset=['Origin', 'Total Deaths', 'Total Affected', 'Total Damages Adj'])

In [ ]:
df_earthquake_impact_origin_pivot = df_earthquake_impact_origin.pivot_table(
    index='Origin',
    columns='Disaster Subtype',
    values=['Total Deaths', 'Total Affected', 'Total Damages Adj'],
    aggfunc='sum',
    fill_value=0
)
df_earthquake_impact_origin_pivot

In [ ]:
df_earthquake_impact_origin_pivot.plot(kind='area', stacked=True)
plt.title('Impact of Earthquakes by Origin')
plt.xlabel('Origin')
plt.ylabel('Total')
plt.xticks(rotation=65)
plt.show()

## Profundización de Análisis de Impacto Temporal con Pivot Tables

#### Por Año

In [ ]:
df_earthquake_impact_year = df_earthquake.dropna(subset=['Year', 'Total Deaths', 'Total Affected', 'Total Damages Adj'])
df_earthquake_impact_year_pivot = df_earthquake_impact_year.pivot_table(
    index='Year',
    columns='Disaster Subtype',
    values=['Total Deaths', 'Total Affected', 'Total Damages Adj'],
    aggfunc='sum',
    fill_value=0
)

df_earthquake_impact_year_pivot

#### Impacto de Terremotos por Año en cada Continente

##### Muertes por Terremotos por continente al año

In [ ]:
plt.figure(figsize=(12,10))
sns.lineplot(data=df_earthquake, x='Start Year', y='Total Deaths', hue='Continent')
plt.ticklabel_format(style='plain', axis='y')

In [ ]:
# Quitar filas con valores NaN en 'Start Year', 'Country', y 'Total Deaths'
df_earthquakes_impact_filtered = df_earthquake.dropna(subset=['Start Year', 'Country', 'Total Deaths'])

# Agrupar los datos filtrados por año y país, y encontrar el conteo máximo de muertes por cada grupo
df_earthquakes_impact_grouped_by_year_country = df_earthquakes_impact_filtered.groupby(['Start Year', 'Country'])['Total Deaths'].agg(['max']).reset_index()

In [ ]:
# Encontrar el país correspondiente para el conteo máximo de muertes en cada año de inicio
max_earthquakes_death_country_by_year = df_earthquakes_impact_grouped_by_year_country.groupby('Start Year').apply(lambda x: x.loc[x['max'].idxmax()])

# Ordenar los resultados por las mayores muertes
max_earthquakes_death_country_by_year = max_earthquakes_death_country_by_year.sort_values(by='max', ascending=False)

# Mostrar los resultados
print("Año con más muertes por país:")
print(max_earthquakes_death_country_by_year[['Start Year', 'Country', 'max']])

In [ ]:
# Ordenar por el máximo conteo de muertes en orden descendente
max_earthquakes_death_country_by_year = max_earthquakes_death_country_by_year.sort_values('max', ascending=False)

# Plot de máximo conteo de muertes por año y país
plt.figure(figsize=(12, 6))
plt.bar(range(len(max_earthquakes_death_country_by_year)), max_earthquakes_death_country_by_year['max'])
plt.xticks(range(len(max_earthquakes_death_country_by_year)), max_earthquakes_death_country_by_year['Start Year'].astype(str) + ' - ' + max_earthquakes_death_country_by_year['Country'], rotation=90)
plt.xlabel('Year - Country')
plt.ylabel('Conteo Máximo de Muertes')
plt.title('Conteo Máximo de Muertes por Year y Country')
plt.tight_layout()

# Show the plot
plt.show()

##### Afectados por terremotos por continente al año

In [ ]:
plt.figure(figsize=(12,10))
sns.lineplot(data=df_earthquake, x='Start Year', y='Total Affected', hue='Continent')
plt.ticklabel_format(style='plain', axis='y')

##### Muertes por terremotos por continente al año

In [ ]:
plt.figure(figsize=(12,10))
sns.lineplot(data=df_earthquake, x='Start Year', y='Total Damages Adj', hue='Continent')
plt.ticklabel_format(style='plain', axis='y')

In [ ]:
# # Agrupar por año y calcular la suma de indicadores de impacto
df_earthquakeimpact_year = df_earthquake.groupby('Year')['Total Deaths', 'Total Affected', 'Total Damages Adj'].sum().reset_index()

# # Plot de análisis temporal para cada indicador de impacto
impact_indicators = ['Total Deaths', 'Total Affected', 'Total Damages Adj']
num_impact = len(impact_indicators)

fig, axes = plt.subplots(num_impact, figsize=(10, 6*num_impact), sharex=True)

for i, indicator in enumerate(impact_indicators):
    ax = axes[i]
    ax.plot(df_earthquakeimpact_year['Year'], df_earthquakeimpact_year[indicator], marker='o')
    ax.set_title(indicator)
    ax.set_xlabel('Año')
    ax.set_ylabel('Total')
    ax.set_xticks(df_earthquakeimpact_year['Year'])
    ax.set_xticklabels(df_earthquakeimpact_year['Year'], rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis Temporal de Muertes Totales
plt.figure(figsize=(10, 6))
plt.plot(df_earthquakeimpact_year.index, df_earthquakeimpact_year['Total Deaths'], marker='o')
plt.title('Análisis Temporal de Muertes Totales en Terremotos')
plt.xlabel('Year')
plt.ylabel('Total Deaths')
plt.xticks(df_earthquakeimpact_year.index, rotation=45, fontsize=8)
plt.yticks(fontsize=8)
plt.show()

# Análisis Temporal de Total de Afectados
plt.figure(figsize=(10, 6))
plt.plot(df_earthquakeimpact_year.index, df_earthquakeimpact_year['Total Affected'], marker='o')
plt.title('Análisis Temporal de Afectados Totales en Terremotos')
plt.xlabel('Year')
plt.ylabel('Total Affected')
plt.xticks(df_earthquakeimpact_year.index, rotation=45, fontsize=8)
plt.yticks(fontsize=8)
plt.show()

# Análisis Temporal de Daños Totales
plt.figure(figsize=(10, 6))
plt.plot(df_earthquakeimpact_year.index, df_earthquakeimpact_year['Total Damages Adj'], marker='o')
plt.title('Análisis Temporal de Daños Totales en Terremotos')
plt.xlabel('Year')
plt.ylabel('Total Damages Adj')
plt.xticks(df_earthquakeimpact_year.index, rotation=45, fontsize=8)
plt.yticks(fontsize=8)
plt.show()


#### Análisis de Años y las Muertes Totales por Terremotos

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_earthquakeimpact_year['Year'], df_earthquakeimpact_year['Total Deaths'], marker='o')
plt.title('Análisis de Años y las Muertes Totales por Terremotos')
plt.xlabel('Año')
plt.ylabel('Muertes Totales')
plt.xticks(df_earthquakeimpact_year['Year'], rotation=45, fontsize=8)
plt.yticks(fontsize=6)
plt.show()

#### Análisis de Años y los Afectados Totales por Terremotos

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_earthquakeimpact_year['Year'], df_earthquakeimpact_year['Total Affected'], marker='o')
plt.title('Análisis de Años y Total de Afectados por Terremotos')
plt.xlabel('Year')
plt.ylabel('Total de Afectados')
plt.xticks(df_earthquakeimpact_year['Year'], rotation=45, fontsize=8)
plt.yticks(fontsize=8)
plt.show()

#### Análisis de Años y los Daños Totales por Terremotos

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_earthquakeimpact_year['Year'], df_earthquakeimpact_year['Total Damages Adj'], marker='o')
plt.title('Análisis de Años y Daños Totales por Terremotos')
plt.xlabel('Año')
plt.ylabel('Daños Totales')
plt.xticks(df_earthquakeimpact_year['Year'], rotation=45, fontsize=8)
plt.yticks(fontsize=8)
plt.show()

Se cuentan 37 años con los diferentes impactos.

### Duración de las Terremotos por Días

In [ ]:
# Calcular la duración para cada evento de inundación
df_earthquake['Duration'] = df_earthquake['End Day'] - df_earthquake['Start Day'] + 1

# Filtrar valores no-finitos en la columna 'Duration'
df_earthquake = df_earthquake[np.isfinite(df_earthquake['Duration'])]

# Convertir columna 'Duration' a integers
df_earthquake['Duration'] = df_earthquake['Duration'].astype(int)

# Histograma del análisis temporal de duración de terremotos
plt.figure(figsize=(10, 6))
plt.hist(df_earthquake['Duration'], bins=range(1, df_earthquake['Duration'].max() + 2), edgecolor='black')
plt.title('Duración de Terremotos por Días')
plt.xlabel('Duración (Días)')
plt.ylabel('Frecuencia')

# Añadir xticks con años
plt.xticks(range(1, df_earthquake['Duration'].max() + 1), fontsize=8)
plt.yticks(fontsize=8)

plt.show()


La mayoría de las terremotos dura máximo de 4 días, de ahí, puede variar entre 2, 3 y 5 días. A lo largo de un mes, la tendencia es que la frecuencia se reduzca, siendo menos probables las terremotos de esta duración.

#### Por Mes

In [ ]:
df_earthquake_impact_startmonth = df_earthquake.dropna(subset=['Start Month', 'End Month', 'Disaster Subtype', 'Total Deaths', 'Total Affected', 'Total Damages Adj'], inplace=False)
df_earthquake_impact_startmonth.isna().sum()

df_earthquake_impact_startmonth_pivot = df_earthquake_impact_startmonth.pivot_table(
    index='Start Month',
    columns='Disaster Subtype',
    values=['Total Deaths', 'Total Affected', 'Total Damages Adj'],
    aggfunc='sum',
    fill_value=0
)

df_earthquake_impact_startmonth_pivot

In [ ]:
# Agrupar por subtipo de inundación y mes
df_earthquake_startmonth = df_earthquake.groupby(['Disaster Subtype', 'Start Month']).size().reset_index(name='Count')

# Plot para cada subtipo de inundación
subtypes = df_earthquake_startmonth['Disaster Subtype'].unique()
num_subtypes = len(subtypes)

fig, axes = plt.subplots(num_subtypes, figsize=(10, 6*num_subtypes), sharex=True)

for i, subtype in enumerate(subtypes):
    data = df_earthquake_startmonth[df_earthquake_startmonth['Disaster Subtype'] == subtype]
    ax = axes[i]
    ax.plot(data['Start Month'], data['Count'], marker='o')
    ax.set_title(subtype)
    ax.set_xlabel('Mes')
    ax.set_ylabel('Frecuencia')
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

plt.tight_layout()
plt.show()


In [ ]:
# Calcular la duración de las terremotos por meses
df_earthquake['Duration (Months)'] = (df_earthquake['End Month'] - df_earthquake['Start Month'] + 1) % 12

# Agrupar por duración y calcular el conteo para cada duración
duration_counts = df_earthquake['Duration (Months)'].value_counts().sort_index()

# Plot de la duración
plt.figure(figsize=(10, 6))
plt.bar(duration_counts.index, duration_counts.values)
plt.title('Duración de las terremotos en Meses')
plt.xlabel('Duración (Meses)')
plt.ylabel('Conteo')
plt.xticks(duration_counts.index)
plt.show()


La mayoría de las terremotos tiene duración de 1 mes, seguida en casi un tercio de las veces por 2 meses. A partir de ahí, la duración disminuye considerablemente entre 3 hasta 6 meses.